In [1]:
# import the required packages
import pandas as pd 
import numpy as np
import random

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, recall_score, precision_score, confusion_matrix

from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

This is a fundrasing company.
- Our goal is to predict individual **Churn rate**.
We will divide this process into two mian parts: feature engineering and modelling

- The frist step is to understand the data, we will use the tables below in this project.
1. **Extrel**: All the donors of the organization                                     
2. **Payhistory**: Payment history of each donor                                         
3. **Communication**: All possible communication between the donor and the organization 
4. **Extrelty**: Description of the activity                                            
5. **Nameaddr**: Socio-demographical information                                        
6. **Commediu**: Description of medium type                                              
7. **Commaint**: Description of the main topic code                                      
8. **Comclas**: Description of the contact class                                        

In [2]:
# import data
comclas = pd.read_csv('./comclas.csv')
commediu = pd.read_csv('./commediu.csv')
communication = pd.read_csv('./communication.csv')
extrel=pd.read_csv('./extrel.csv')
extrelty = pd.read_csv('./extrelty.csv')
nameaddr = pd.read_csv("./nameaddr.csv")
payhistory = pd.read_csv('./payhistory.csv')

## Explore and clean data
- inspire the table
- convert datatype

1.Table: comclas

In [3]:
# comclas : Description of the contact class
print(comclas.dtypes)
comclas.head()

CLASCODE    object
CLASDESC    object
dtype: object


,CLASCODE,CLASDESC
0,CT,CTS-item
1,RI,Request for information
2,CM,Complaint
3,TA,Threath / Accusation
4,AK,Acknowledgement


2. Table: commediu

In [4]:
# commediu : Description of medium type
print(commediu.dtypes)
commediu.head()

MEDIUMCODE    object
MEDIUMDESC    object
dtype: object


,MEDIUMCODE,MEDIUMDESC
0,FF,Face to Face contact
1,FX,Fax
2,TX,Telex
3,EM,Electronic Mail
4,TE,Telephone call


3. Table: communication

In [5]:
#communication :All possible communication between the donor and the organization
print(communication.dtypes)
communication.head()

CONTID         int64
MEDIUMCODE    object
MNTOPCODE     object
CLASCODE      object
EXTRELNO       int64
CONTDIREC     object
CONTDATE      object
dtype: object


,CONTID,MEDIUMCODE,MNTOPCODE,CLASCODE,EXTRELNO,CONTDIREC,CONTDATE
0,405402,CI,CTS,CT,20,O,2007-05-06
1,430904,CI,CTS,CT,20,O,2007-06-11
2,592732,CI,CTS,CT,20,O,2007-10-18
3,593375,CI,CTS,CT,20,O,2007-10-18
4,618514,CI,CTS,CT,20,O,2007-11-25


In [6]:
# Change the the datetime type
communication['CONTDATE']=pd.to_datetime(communication['CONTDATE'],format='%Y-%m-%d')
print(communication.dtypes)

CONTID                 int64
MEDIUMCODE            object
MNTOPCODE             object
CLASCODE              object
EXTRELNO               int64
CONTDIREC             object
CONTDATE      datetime64[ns]
dtype: object


4. Table: extrel

In [7]:
#extrel  : All the donors of the organization
print(extrel.dtypes)
extrel.head()

EXTRELNO       int64
EXRELACTCD    object
EXTRELSTDT    object
EXRELDATEN    object
dtype: object


,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN
0,26414,CT,2008-01-30,NaT
1,26419,FP,2005-02-26,NaT
2,26424,FP,2005-02-26,2009-12-21
3,26430,FP,2005-02-26,2019-01-28
4,26430,CT,2010-03-04,NaT


In [8]:
# Change the datetime
extrel['EXTRELSTDT'] = pd.to_datetime(extrel['EXTRELSTDT'], format='%Y-%m-%d')
extrel['EXRELDATEN'] = pd.to_datetime(extrel['EXRELDATEN'], format='%Y-%m-%d')
print(extrel.dtypes)

# check the earliest and latest register time
print('The earliest time:',min(extrel['EXTRELSTDT']))
print('The latest time:',max(extrel['EXTRELSTDT']))

# check the earliest and latest end contract time
print('The earliest time:',min(extrel['EXRELDATEN']))
print('The latest time:',max(extrel['EXRELDATEN']))

EXTRELNO               int64
EXRELACTCD            object
EXTRELSTDT    datetime64[ns]
EXRELDATEN    datetime64[ns]
dtype: object
The earliest time: 2002-10-29 00:00:00
The latest time: 2020-01-29 00:00:00
The earliest time: NaT
The latest time: NaT


5. Table: extrelty

In [9]:
#extrelty : Description of the activity
print(extrelty.dtypes)
extrelty.head()

EXRELACTCD    object
EXRELACTDE    object
dtype: object


,EXRELACTCD,EXRELACTDE
0,FP,CommitmentDo
1,PS,Prospect
2,PF,Prosp Fundra
3,PP,Prosp Promot
4,GR,Grantor


6. Table: nameaddr

In [10]:
#nameaddr : Socio-demographical information
print(nameaddr.dtypes)
nameaddr.head()

EXTRELNO       int64
NAME1TITLE    object
POSTCODE      object
LANGUACODE    object
dtype: object


,EXTRELNO,NAME1TITLE,POSTCODE,LANGUACODE
0,19,HR,2500,NL
1,20,HR,9550,NL
2,35,HM,2150,NL
3,42,FA,3012,NL
4,43,MW,3000,NL


7. Table: payhistory

In [11]:
#payhistory : Payment history of each donor
print(payhistory.dtypes)
payhistory.head()

PID            int64
PDATE         object
PAMT         float64
EXTRELNO       int64
PAYTYPECD     object
STATUS        object
dtype: object


,PID,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS
0,38,2006-12-28,9.57,19,X,CO
1,39,2006-12-28,41.32,20,X,CO
2,40,2006-12-28,13.10,20,X,CO
3,54091,2006-12-29,2.02,20,X,CO
4,104480,2007-05-16,0.50,20,D,OK


In [12]:
# Change the datetime 
payhistory['PDATE']=pd.to_datetime(payhistory['PDATE'],format='%Y-%m-%d')
print(payhistory.dtypes)
# check the earliest and latest time
print('The earliest time:',min(payhistory['PDATE']))
print('The latest time:',max(payhistory['PDATE']))

PID                   int64
PDATE        datetime64[ns]
PAMT                float64
EXTRELNO              int64
PAYTYPECD            object
STATUS               object
dtype: object
The earliest time: 2006-12-28 00:00:00
The latest time: 2020-01-21 00:00:00


## Feature engineering

1. Define the time window
2. Define the active customers
3. Define the churn customers
4. Build up some useful features

### 1. Define the time window

- Assume the churn marketing campaign will be implement in the entire year(2025/01/01 - 2025/12/31)
- We build up the model today(2024/10/30) and the past payhistory data is from 2006-12-08 to 2020-01-21.


Implementing Purpose Model

----------INDEPENDENT---------|xxxxGAPxxxxxx|----------DEPENDENT---------|
-                       30/10/2024       01/01/2025              31/12/2025

Model Building

------------INDEPENDENT----------|xxxxGAPxxxxxx|------DEPENDENT-------|
-                          30/10/2018    01/01/2019             31/12/2019


In [13]:
# Define the time window (model building)
end_independent = datetime.strptime('30/10/2018', '%d/%m/%Y')
start_dependent = datetime.strptime('01/01/2019', '%d/%m/%Y')
end_dependent = datetime.strptime('31/12/2019', '%d/%m/%Y')

### 2. Define the active customers

In [14]:
# "EXTRELNO" = userid
# define the active customers who started contract before the end of independent period and ended after dependent period or haven't ended yet.
active_customers = extrel[(extrel['EXTRELSTDT']<=end_independent) & (((extrel['EXRELDATEN']>=start_dependent)&(extrel['EXRELDATEN']<=end_dependent))|(extrel['EXRELDATEN'].isnull()))]
active_customers.head()

,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN
0,26414,CT,2008-01-30,NaT
1,26419,FP,2005-02-26,NaT
3,26430,FP,2005-02-26,2019-01-28
4,26430,CT,2010-03-04,NaT
5,26431,FP,2005-02-26,NaT


### 3. Define the Churn Customers

In [15]:
# The custoemr who ended the contract during the dependent period
# If "EXRELDATEN" is null, means this customer has not ended the contract yet. 
# Churn indicator 1, Non-churn indicator 0
active_customers["Churn"] = np.where(((active_customers["EXRELDATEN"].isnull())|(active_customers["EXRELDATEN"]>end_dependent)),0,1)
active_customers.head()

,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN,Churn
0,26414,CT,2008-01-30,NaT,0
1,26419,FP,2005-02-26,NaT,0
3,26430,FP,2005-02-26,2019-01-28,1
4,26430,CT,2010-03-04,NaT,0
5,26431,FP,2005-02-26,NaT,0


### 4. Building independent variables

(1) Transaction table: payhistory

RMF

In [16]:
# RMF

# extract the data corresponding to the time window 
pay_indep = payhistory[(payhistory['PDATE']<=end_independent)&(payhistory['PAMT']>0)]
pay_dep = payhistory[((payhistory['PDATE']<=end_dependent)&(payhistory['PDATE']>=start_dependent))&(payhistory['PAMT']>0)]

# Recency
# recency = today-the last purchased date. Here we image that today is end of independent day(the day we started to build our model)
Recency = (end_independent-pay_indep[['EXTRELNO','PDATE']].groupby('EXTRELNO')['PDATE'].max()).reset_index().rename(columns = {'PDATE':'Recency'})

# Monetary
# Monetary contains total monetary and average monetary
Monetary = pay_indep[['EXTRELNO','PAMT']].groupby('EXTRELNO')['PAMT'].agg(['sum','mean']).reset_index().rename(columns={'sum':'Total_Monetary','mean':'Average_Monetary'})

# Frequency
Frequency = pay_indep[['EXTRELNO']].value_counts().reset_index().rename(columns={'count':'Frequency'})

PAYTYPECD

- O Bank transfer
- D Permanent order
- E Own initiative
- X Unknown

In [17]:
# check the null value in paytypecd
print(pay_indep.isnull().sum())

# fill in the null value as unknown type
pay_indep['PAYTYPECD'] = pay_indep['PAYTYPECD'].fillna('X')

# Get dummy values 
paytype = pd.get_dummies(pay_indep[['EXTRELNO','PAYTYPECD']],columns=['PAYTYPECD']).reset_index().rename(columns={'PAYTYPECD_D':'used_order','PAYTYPECD_E':'used_initiative','PAYTYPECD_O':'used_bank','PAYTYPECD_X':'used_unknown'}).drop(columns=['index'])

# groupby userid and get the frequency of each payment type
paytype = paytype.groupby('EXTRELNO').sum().reset_index()

paytype.head()

PID              0
PDATE            0
PAMT             0
EXTRELNO         0
PAYTYPECD    10648
STATUS           1
dtype: int64


,EXTRELNO,used_order,used_initiative,used_bank,used_unknown
0,19,0,0,0,1
1,20,138,0,0,4
2,35,138,0,0,2
3,42,81,0,0,3
4,43,221,0,0,6


(2) Communication table

In [18]:
# split the table based on the independent and dependent period
communication_indep = communication[communication['CONTDATE']<=end_independent]
communication_dep = communication[(communication['CONTDATE']>=start_dependent)&(communication['CONTDATE']<=end_dependent)]

MEDIUMCODE : Medium of the contact
- MEDIUMCODE description is in the commediu table

In [19]:
# inspire commediu table to see any interesting thing
print(commediu)

   MEDIUMCODE                     MEDIUMDESC
0          FF           Face to Face contact
1          FX                            Fax
2          TX                          Telex
3          EM                Electronic Mail
4          TE                 Telephone call
5          ES                   Express mail
6          AM              Answering machine
7          RM  Regular mail (CTS-items only)
8          AF               Application form
9          LE                        Letters
10         ME                           Memo
11         CA         Change of address card
12         AC           Advertisement coupon
13         MR                  Mail returned
14        NaN              Newspaper article
15         TM                     Translator


In [73]:
# Based on description we can summary the communication mediums into three categories.
# a. electronic (FX,TX,EM,TE,AM,TM)
# b. face to face (FF)
# c. papers(ES,RM,AF,LE,ME,CA,AC,MR,NaN)
# d. CI is unknown

# extract medium data
medium = communication_indep[['EXTRELNO','MEDIUMCODE']]

# define a category function
def medium_category(a):
    if a in ["FX","TX","EM","TE","AM","TM"]:
        return 'electronic'
    elif a in ["FF"]:
        return 'face_to_face'
    elif a in ["ES","RM","AF","LE","ME","CA","AC","MR","NaN"]:
        return'paper'
    else:
        return 'unknown'

# Apply the function in mediumcode
medium["medium_cat"]=medium["MEDIUMCODE"].apply(lambda x:medium_category(x))

# get dummy variables in medium
medium_cat = pd.get_dummies(medium[['EXTRELNO','medium_cat']],columns=['medium_cat']).reset_index().groupby('EXTRELNO').sum().reset_index().drop(columns=['index'])

medium_cat.head()

,EXTRELNO,medium_cat_electronic,medium_cat_face_to_face,medium_cat_paper,medium_cat_unknown
0,20,1,0,18,30
1,35,1,0,21,34
2,42,0,0,12,24
3,43,4,0,25,83
4,60,1,0,13,22


CLASCODE : Class of the contact
- decription in the comclas table

In [21]:
# show
print(comclas)

# Here we mainly identify complaint CM

  CLASCODE                 CLASDESC
0       CT                 CTS-item
1       RI  Request for information
2       CM                Complaint
3       TA     Threath / Accusation
4       AK          Acknowledgement
5       RM                 Reminder
6       ID     Information delivery
7       AN                   Answer


In [22]:
# extract the data set
complaint = communication_indep[['EXTRELNO','CLASCODE']]
# set indicator
complaint['complaint']=np.where(complaint['CLASCODE']=='CM',1,0)
# calculate the number of complaint
complaint = complaint.groupby('EXTRELNO')['complaint'].sum().reset_index()
# show
complaint.head()

,EXTRELNO,complaint
0,20,0
1,35,0
2,42,0
3,43,0
4,60,0


CONTDIREC: Direction of the communication
- I Incoming
- P Outgoing

In [23]:
direction = communication_indep[['EXTRELNO','CONTDIREC']]
direction = pd.get_dummies(direction,columns=['CONTDIREC']).reset_index().groupby('EXTRELNO').sum().reset_index().drop(columns=['index']).rename(columns={'CONTDIREC_I':'Incoming','CONTDIREC_O':'Outgoing'})
direction.head()

,EXTRELNO,Incoming,Outgoing
0,20,0,49
1,35,1,55
2,42,1,35
3,43,4,108
4,60,2,34


Communication times

In [24]:
# communication times
comunication_num = communication_indep[['EXTRELNO']].value_counts().reset_index().rename(columns={'count':'comu_count'})
comunication_num.head()

,EXTRELNO,comu_count
0,181194,707
1,1773,544
2,25455,473
3,35750,434
4,35231,373


(3) User table: Extrel
- Nameaddr: Socio-demographical information

In [26]:
print(nameaddr)

       EXTRELNO NAME1TITLE POSTCODE LANGUACODE
0            19         HR     2500         NL
1            20         HR     9550         NL
2            35         HM     2150         NL
3            42         FA     3012         NL
4            43         MW     3000         NL
...         ...        ...      ...        ...
24181    245025         MW     1500         NL
24182    245029         MW     9810         NL
24183    245032         MW     2940         NL
24184    245046         FA     9470         NL
24185    245048         MW     8700         NL

[24186 rows x 4 columns]


Language

In [74]:
#check unique language
nameaddr['LANGUACODE'].unique()

# extract language table
language = nameaddr[['EXTRELNO','LANGUACODE']]

Postcode


In [71]:
# extract postcode
postcode = nameaddr[['EXTRELNO','POSTCODE']]

# extract the first two digits
postcode['first_digits'] = postcode['POSTCODE'].str[0:2]


In [75]:
# Define the function to determine the province based on postcode
def get_province(a):
    try:
        # Convert to integer
        a = int(a)
    except (ValueError, TypeError):
        # If conversion fails, return 'unknown'
        return 'unknown'
    
    # Check ranges to determine the province
    if a in range(11, 12):
        return 'Brussels Capital Region'
    elif a in [13, 14]:
        return 'Walloon Brabant'
    elif a in list(range(15, 19)) + list(range(30, 34)):
        return 'Flemish Brabant'
    elif a in range(20, 29):
        return 'Antwerp'
    elif a in range(35, 39):
        return 'Limburg'
    elif a in range(40, 49):
        return 'Liège'
    elif a in range(50, 59):
        return 'Namur'
    elif a in list(range(60, 65)) + list(range(70, 79)):
        return 'Hainaut'
    elif a in range(66, 69):
        return 'Luxembourg'
    elif a in range(80, 89):
        return 'West Flanders'
    elif a in range(90, 99):
        return 'East Flanders'
    else:
        return 'unknown'  # Handle cases where postcode doesn't match any region

# Apply the function to determine the province
postcode['province'] = postcode['first_digits'].apply(get_province)

# drop columns
postcode = postcode.drop(columns=['first_digits','POSTCODE'])

Basetable

In [80]:
# merge all feature tables
basetable = active_customers[['EXTRELNO','Churn']].merge(
                                      Recency, how='inner',on='EXTRELNO').merge(
                                      Monetary,how='inner',on='EXTRELNO').merge(
                                      Frequency,how='inner',on='EXTRELNO').merge(
                                      paytype,how='inner',on='EXTRELNO').merge(
                                      medium_cat,how='inner',on='EXTRELNO').merge(
                                      complaint,how='inner',on='EXTRELNO').merge(
                                      direction,how='inner',on='EXTRELNO').merge(
                                      comunication_num,how='inner',on='EXTRELNO').merge(
                                      language,how='inner',on='EXTRELNO').merge(
                                      postcode, how='inner',on='EXTRELNO')

#show
basetable.head()

# save the basetable
basetable.to_csv('./basetable_churn.csv')